## Introduction

There a many ways how to approach programming a quantum computer. The two extreme cases are
* manual development of an algorithm for each problem. This is similar to how code for HPC is developed, highly customized and highly specialized for each problem with high effort
* use a high level compiler to transform an abstract high level description of the problem to a machine executable form. One example of that is Mathematical Programming which is implemented by IBM CPLEX classically (so far only convex problems).

Let's use here what Qiskit provides since the Aqua 0.7.0 release:

Qiskit introduces the `QuadraticProgram` class to make a model of an optimization problem.
More precicely, it deals with quadratically constrained quadratic programs given as follows:

$$
\begin{align}
\text{minimize}\quad& x^\top Q_0 x + c^\top x\\
\text{subject to}\quad& A x \leq b\\
& x^\top Q_i x + a_i^\top x \leq r_i, \quad 1,\dots,i,\dots,q\\
& l_i \leq x_i \leq u_i, \quad 1,\dots,i,\dots,n,
\end{align}
$$

where the $Q_i$ are $n \times n$ matrices, $A$ is a $m \times n$ matrix , $x$, and $c$ are $n$-dimensional vectors, $b$ is an $m$-dimensional vector, and where $x$ can defined as binary, integer, or continuous variables.
In addition to "$\leq$" constraints 'QuadraticProgram' also supports "$\geq$" and "$=$".

In [1]:
# DOcplex
from docplex.mp.model import Model
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToIsing, InequalityToEquality, IntegerToBinary, LinearEqualityToPenalty

# Loading your IBM Q account(s)
from qiskit import IBMQ
#provider = IBMQ.load_account()
#provider = IBMQ.get_provider(hub='ibm-q')

## Example

Here we use bin packing as an example problem. It's intentionally kept small and simple to concentrate on the workflow below.

Just insert any other more complex model here and reuse the transformation and solver steps below.

In [2]:
# Create an instance of a model and variables
mdl = Model(name='bin_pack')
x = {i: mdl.binary_var(name='x_{0}'.format(i)) for i in range(1,4)}

# Objective function
max_vars_func = mdl.sum(x[i] for i in range(1,4))
mdl.maximize(max_vars_func)

# Constraints
weight = [2, 4, 6]
mdl.add_constraint(mdl.sum(weight[i-1]*x[i] for i in range(1,4)) == 10)

print(mdl.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: bin_pack

Maximize
 obj: x_1 + x_2 + x_3
Subject To
 c1: 2 x_1 + 4 x_2 + 6 x_3 = 10

Bounds
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1

Binaries
 x_1 x_2 x_3
End



In [3]:
# load QuadraticProgram from a Docplex model
qp = QuadraticProgram()
qp.from_docplex(mdl)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: bin_pack

Maximize
 obj: x_1 + x_2 + x_3
Subject To
 c0: 2 x_1 + 4 x_2 + 6 x_3 = 10

Bounds
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1

Binaries
 x_1 x_2 x_3
End



In [4]:
# convert inequalities to equalities using slack variables
ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.encode(qp)
print(qp_eq.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: bin_pack

Maximize
 obj: x_1 + x_2 + x_3
Subject To
 c0: 2 x_1 + 4 x_2 + 6 x_3 = 10

Bounds
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1

Binaries
 x_1 x_2 x_3
End



In [5]:
# move all conditions to the objective function ("penalty")
lineq2penalty = LinearEqualityToPenalty()
qubo = lineq2penalty.encode(qp_eq)
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: bin_pack

Maximize
 obj: 4000001 x_1 + 8000001 x_2 + 12000001 x_3 + [ - 800000 x_1^2
      - 3200000 x_1*x_2 - 4800000 x_1*x_3 - 3200000 x_2^2 - 9600000 x_2*x_3
      - 7200000 x_3^2 ]/2 -10000000
Subject To

Bounds
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1

Binaries
 x_1 x_2 x_3
End



## Solvers

Now that we finally prepared the Quadratic Program for the problem at hand, we have several options to solve it.

One family of solvers is reused from other quantum computing problem domains, in the case of VQE (Variational Quantum Eigensolver) and QAOA (Quantum Approximate Optimization Algorithm) mostly quantum chemistry.
The idea here is to convert the Quadratic Program into an Ising Hamiltonian and find the minimum Eigenvector.
That Eigenvector represents the optimal solution to the Quadratic Program.

Another recent family is based on Grover Search. GAS (Grover Adaptive Search) is provided in the form of `GroverOptimizer`.

An extension to binary 'QUBO' problems is the mixed-binary problem class called 'MBCO'. See "3-ADMM-H" for details.

In [6]:
# convert to Ising Hamiltonian to be able to search for minimum eigenvector later
qp2op = QuadraticProgramToIsing()
op, offset = qp2op.encode(qubo)
print('offset: {}'.format(offset))
print('operator:')
print(op.print_details())

offset: 2999998.5
operator:
IIZ	(800000.5+0j)
IZI	(1600000.5+0j)
ZII	(2400000.5+0j)
IZZ	(400000+0j)
ZIZ	(600000+0j)
ZZI	(1200000+0j)



In [7]:
from qiskit import BasicAer
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.aqua.algorithms import QAOA, VQE, ExactEigensolver, NumPyMinimumEigensolver

vqe_mes = VQE(quantum_instance=BasicAer.get_backend('statevector_simulator'))
qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator'))
exact_mes = NumPyMinimumEigensolver()

vqe = MinimumEigenOptimizer(vqe_mes)   # using VQE
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [8]:
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

x=[0.0,1.0,1.0], fval=2.0


In [9]:
exact_result = exact.solve(qubo)
print(exact_result)

x=[0.0,1.0,1.0], fval=2.0


In [10]:
#vqe_result = vqe.solve(qubo)
#print(vqe_result)

## Improvements

### convenience: 

The usability and convenience has been improved lately by supporting models with
* linear and quadratic equations
* equalities and inequalities
* binary and integer variables

Most of these improvements of scope have been paid for in the number of qubits (slack variables, int approximation).
This makes the representation less compact and with that less fit for near term Quantum Computers.


### depth:

Recursive solvers are introduced to limit the Quantum Circuit depth.
The circuit is iteratively refined and executed many times.
Qiskit provides this via `RecursiveMinimumEigenOptimizer`.

### scope: 

* Binary optimization (QUBO): On top of Ising based solvers, Grover based solvers (`GroverOptimizer`) have been recently introduced.
* Mixed-binary optimization (MBCO): The ADMM Optimizer can solve classes of mixed-binary constrained optimization problems
* Mixed-integer methods are still missing.

### width:

Future research is needed on hybrid solvers that process hard and compact subproblems on the Quantum Computer and combine this via classical solvers to the final solution.


In [11]:
# Example of the usage of the recursive solver
rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)
rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)

x=[0.0,1.0,1.0], fval=2.0


In [12]:
# Example using GAS `GroverOptimizer`
from qiskit.optimization.algorithms import GroverOptimizer
backend = BasicAer.get_backend('statevector_simulator')
grover_optimizer = GroverOptimizer(6, num_iterations=10, quantum_instance=backend)
results = grover_optimizer.solve(qubo)
print("x={}".format(results.x))
print("fval={}".format(results.fval))

x=[0.0, 0.0, 0.0]
fval=-10000000.0


In [13]:
#IBMQ.providers()    # List all available providers
#IBMQ.get_provider(hub='ibm-q')
#provider = IBMQ.get_provider(hub='ibm-q')
#provider.backends()

In [14]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright